<a href="https://colab.research.google.com/github/Champioa00/LAB1/blob/master/YESSIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [189]:
## LAB 2 ##
## Anthony Champion ##
## IST 718 ##

In [190]:
## Load required packages ##
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
#from botocore.client import Config
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from fbprophet import Prophet
import statsmodels.api as sm
from  multiprocessing import Pool, cpu_count
import pandas as pd
import numpy as np
#import ibm_boto3
import itertools
import types
import math
import time

sns.set(rc={'figure.figsize':(11.7,8.27)})
%matplotlib inline 

DATA Cleaning:

Let's start with importing the difffernt data sets.  From there we can explore and perform the needed data cleansing. 



In [191]:
df_data_1 = pd.read_csv('Zip_zhvi_uc_sfr_month.csv')
df_data_1.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01,1996-02,1996-03,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,1997-07,1997-08,1997-09,1997-10,1997-11,1997-12,1998-01,1998-02,1998-03,1998-04,1998-05,1998-06,1998-07,...,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.792330e+06,2.687480e+06,2.750319e+06,2.745687e+06,2.672237e+06,2.703466e+06,2.799758e+06,2.762261e+06,2.972422e+06,2.964929e+06,2.941399e+06,2.770281e+06,2.709474e+06,2.807152e+06,2.757996e+06,2.898636e+06,2.883489e+06,2.813502e+06,2.835291e+06,2.864990e+06,2.710262e+06,2.851837e+06,2.915106e+06,2.778842e+06,2.812505e+06,2.801768e+06,2.872429e+06,2869819.021,2.792227e+06,2.880981e+06,2.936416e+06,3.033201e+06,2.904437e+06,2.801494e+06,2.721834e+06,2.640603e+06,2.616576e+06,2.600348e+06,2.563603e+06,2.729760e+06
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,327184.4715,327713.4837,325984.4579,332778.2610,334414.5751,337601.7483,337856.7732,340445.1863,346222.3082,348798.2996,353450.8352,352990.2028,354387.5651,357231.4176,359681.4492,361963.9914,372552.6507,369575.8297,373462.4970,348818.2141,341442.7292,323433.5647,315459.2506,316542.5223,295871.2783,300335.2839,300666.0235,309593.5198,315575.7585,329702.7681,340326.4415,...,7.892213e+05,7.910212e+05,7.899114e+05,7.805394e+05,7.872967e+05,7.919914e+05,7.900941e+05,7.925490e+05,7.910878e+05,7.864774e+05,7.838324e+05,7.801943e+05,7.750737e+05,7.740944e+05,7.725794e+05,7.801559e+05,7.915183e+05,7.910302e+05,7.915584e+05,7.939067e+05,7.876053e+05,7.908220e+05,7.917972e+05,7.999406e+05,8.017769e+05,7.938478e+05,7.989722e+05,797140.864,8.138223e+05,8.136838e+05,8.204414e+05,8.171122e+05,8.237994e+05,8.301982e+05,8.286601e+05,8.252674e+05,8.381829e+05,8.389180e+05,8.381373e+05,8.495258e+05
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.306724e+06,5.303907e+06,5.307117e+06,5.156562e+06,5.249348e+06,5.182788e+06,5.058869e+06,5.166357e+06,5.043388e+06,5.041397e+06,4.900863e+06,4.846561e+06,4.806427e+06,4.780078e+06,4.760010e+06,4.874589e+06,4.783694e+06,4.670615e+06,4.821009e+06,4.831602e+06,4.917240e+06,4.823658e+06,4.681429e+06,4.742684e+06,4.750099e+06,4.737670e+06,4.734036e+06,4503091.435,4.820726e+06,5.032885e+06,5.072752e+06,5.104681e+06,4.991198e+06,5.204816e+06,5.430261e+06,5.489555e+06,5.183702e+06,5.143927e+06,5.115112e+06,5.111999e+06
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,178597.7329,182239.3716,181934.9870,179668.4547,184266.4870,183117.2403,188706.1648,183779.9983,186036.3819,182704.3914,184768.1298,187562.7857,189633.2751,189402.0634,193389.9616,199159.9068,199053.6127,201124.2151,205113.1101,209095.7221,211203.2530,207095.6308,207235.3069,204503.8534,203883.6108,205334.1647,203134.1089,203749.4133,203032.1247,203485.1372,202113.1230,...,3.530590e+05,3.541628e+05,3.525020e+05,3.554251e+05,3.556854e+05,3.553540e+05,3.555060e+05,3.568726e+05,3.563078e+05,3.568684e+05,3.551285e+05,3.563217e+05,3.550922e+05,3.540023e+05,3.537106e+05,3.547125e+05,3.549502e+05,3.582089e+05,3.608014e+05,3.609959e+05,3.610848e+05,3.610934e+05,3.634250e+05,3.641847e+05,3.671116e+05,3.720254e+05,3.768957e+05,380199.342,3.884185e+05,3.970397e+05,4.039702e+05,4.138433e+05,4.270610e+05,4.386916e+05,4.466750e+05,4.535979e+05,4.605603e+05

In [ ]:
list(na_values.index)

In [ ]:
na_values = df_data_1.isna().sum()
list(na_values)

In [192]:
## Check to see if the data was loaded correctly ##
na_values = df_data_1.isna().sum()
print(na_values)

RegionID       0
SizeRank       0
RegionName     0
RegionType     0
StateName      0
              ..
2021-09       60
2021-10       44
2021-11       26
2021-12        0
2022-01        0
Length: 322, dtype: int64


In [198]:
## Load in metro data from the US Department of Labor ##
metro_area = pd.read_csv('OD.csv')
metro_area.head()


,ZIP CODE,STATE,MSA No.,GPCI_W,GPCI_PE,GPCI_ME,County No.,MSA Name
0,210,NH,14460.0,1.0,1.046,0.658,15.0,"Boston-Cambridge-Quincy, MA-NH MSA"
1,211,NH,14460.0,1.0,1.046,0.658,15.0,"Boston-Cambridge-Quincy, MA-NH MSA"
2,212,NH,14460.0,1.0,1.046,0.658,15.0,"Boston-Cambridge-Quincy, MA-NH MSA"
3,213,NH,14460.0,1.0,1.046,0.658,15.0,"Boston-Cambridge-Quincy, MA-NH MSA"
4,214,NH,14460.0,1.0,1.046,0.658,15.0,"Boston-Cambridge-Quincy, MA-NH MSA"


In [201]:
# Keep data with missing metro name. 
missing_metro = df_data_1[df_data_1['Metro'].isna()]
missing_index = missing_metro.index

# Drop observation without Metro data. These observations will then be reattached
df_metro_complete = df_data_1.drop(index=missing_index)

# Keep zip code and msa name from the metro_area data set
metro_area_simp = metro_area[['ZIP CODE', 'MSA Name']]

# Merge missing metro with metro_area_simp 
metro_complete = missing_metro.merge(metro_area_simp, how='left', left_on='RegionName', right_on='ZIP CODE')

In [233]:
# Check to see that there are no NaN variables in the new column. 
print(metro_complete['MSA Name'].isna().sum())

# There are no NaN values now. We'll now replace the Metro column with the MSA Name column, drop the overlapping columns, and 
# reattach the observations to the df_data_1 data set. 
metro_complete['Metro'] = metro_complete['MSA Name']
metro_complete1 = metro_complete.iloc[:, 0:322]

# Reattach data to original data. 
df_data_1 = df_metro_complete.append(metro_complete1)
print(df_data_1.shape)

0
(31990, 322)


In [235]:
from tqdm import tqdm

In [241]:
df_data_2

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01,1996-02,1996-03,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,1997-07,1997-08,1997-09,1997-10,1997-11,1997-12,1998-01,1998-02,1998-03,1998-04,1998-05,1998-06,1998-07,...,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,228165.313018,233969.358296,233133.064945,233443.970861,237353.998041,237486.585262,238150.921270,239538.443050,239793.729378,240699.035363,241930.896414,242230.916226,243306.330079,243374.943609,244168.123100,244800.614066,246075.490814,247681.544210,250038.248818,251547.997078,251334.345944,252624.571633,256025.444268,257593.000051,259983.312856,259983.694731,260806.926778,262810.475357,264848.773670,266633.813405,268861.124317,...,2.792330e+06,2.687480e+06,2.750319e+06,2.745687e+06,2.672237e+06,2.703466e+06,2.799758e+06,2.762261e+06,2.972422e+06,2.964929e+06,2.941399e+06,2.770281e+06,2.709474e+06,2.807152e+06,2.757996e+06,2.898636e+06,2.883489e+06,2.813502e+06,2.835291e+06,2.864990e+06,2.710262e+06,2.851837e+06,2.915106e+06,2.778842e+06,2.812505e+06,2.801768e+06,2.872429e+06,2.869819e+06,2.792227e+06,2.880981e+06,2.936416e+06,3.033201e+06,2.904437e+06,2.801494e+06,2.721834e+06,2.640603e+06,2.616576e+06,2.600348e+06,2.563603e+06,2.729760e+06
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,327184.471500,327713.483700,325984.457900,332778.261000,334414.575100,337601.748300,337856.773200,340445.186300,346222.308200,348798.299600,353450.835200,352990.202800,354387.565100,357231.417600,359681.449200,361963.991400,372552.650700,369575.829700,373462.497000,348818.214100,341442.729200,323433.564700,315459.250600,316542.522300,295871.278300,300335.283900,300666.023500,309593.519800,315575.758500,329702.768100,340326.441500,...,7.892213e+05,7.910212e+05,7.899114e+05,7.805394e+05,7.872967e+05,7.919914e+05,7.900941e+05,7.925490e+05,7.910878e+05,7.864774e+05,7.838324e+05,7.801943e+05,7.750737e+05,7.740944e+05,7.725794e+05,7.801559e+05,7.915183e+05,7.910302e+05,7.915584e+05,7.939067e+05,7.876053e+05,7.908220e+05,7.917972e+05,7.999406e+05,8.017769e+05,7.938478e+05,7.989722e+05,7.971409e+05,8.138223e+05,8.136838e+05,8.204414e+05,8.171122e+05,8.237994e+05,8.301982e+05,8.286601e+05,8.252674e+05,8.381829e+05,8.389180e+05,8.381373e+05,8.495258e+05
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,228165.313018,233969.358296,233133.064945,233443.970861,237353.998041,237486.585262,238150.921270,239538.443050,239793.729378,240699.035363,241930.896414,242230.916226,243306.330079,243374.943609,244168.123100,244800.614066,246075.490814,247681.544210,250038.248818,251547.997078,251334.345944,252624.571633,256025.444268,257593.000051,259983.312856,259983.694731,260806.926778,262810.475357,264848.773670,266633.813405,268861.124317,...,5.306724e+06,5.303907e+06,5.307117e+06,5.156562e+06,5.249348e+06,5.182788e+06,5.058869e+06,5.166357e+06,5.043388e+06,5.041397e+06,4.900863e+06,4.846561e+06,4.806427e+06,4.780078e+06,4.760010e+06,4.874589e+06,4.783694e+06,4.670615e+06,4.821009e+06,4.831602e+06,4.917240e+06,4.823658e+06,4.681429e+06,4.742684e+06,4.750099e+06,4.737670e+06,4.734036e+06,4.503091e+06,4.820726e+06,5.032885e+06,5.072752e+06,5.104681e+06,4.991198e+06,5.204816e+06,5.430261e+06,5.489555e+06,5.183702e+06,5.143927e+06,5.115112e+06,5.111999e+06
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,178597.732900,182239.371600,181934.987000,179668.454700,184266.487000,183117.240300,188706.164800,183779.998300,186036.381900,182704.391400,184768.129800,187562.785700,

In [269]:
list(test.iloc[:,0:9].columns)

['RegionID',
 'SizeRank',
 'RegionName',
 'RegionType',
 'StateName',
 'State',
 'City',
 'Metro',
 'CountyName']

In [274]:
test = df_data_2.dropna()
final_df = pd.melt(test, id_vars=list(test.iloc[:,0:9].columns), value_vars=list(test.iloc[:,9:].columns))
final_df
final_df['variable'] = pd.to_datetime(final_df['variable'], infer_datetime_format=True)
print(final_df.shape)
print(final_df.dtypes)


(8410623, 11)
RegionID               int64
SizeRank               int64
RegionName             int64
RegionType            object
StateName             object
State                 object
City                  object
Metro                 object
CountyName            object
variable      datetime64[ns]
value                float64
dtype: object


In [ ]:
# Obtain column names of year-month combinations. 
year_month = df_data_1.iloc[:, 9:].columns.values

df_data_2 = df_data_1

# Iterate over year-month combination, replacing NaN with average value for metropolitan area 
for ym in tqdm(df_data_2[year_month]):
  print(ym)
  df_data_2[ym] = df_data_2.groupby('Metro')[ym].transform(lambda x: x.fillna(x.mean()))

In [219]:
df_data_2[ym]

0       2.729760e+06
1       8.495258e+05
2       5.111999e+06
3       4.786334e+05
4       1.141322e+06
            ...     
7920    2.178315e+05
7921    1.060980e+05
7922    5.254660e+05
7923    1.047266e+05
7924    2.633532e+05
Name: 2022-01, Length: 31990, dtype: float64

In [234]:
df_data_1.groupby('Metro')[ym].transform(lambda x: x)[df_data_1.groupby('Metro')[ym].transform(lambda x: x).isna()]

Series([], Name: 2022-01, dtype: float64)

In [227]:
df_data_2.groupby('Metro')[ym].transform(lambda x: x.fillna(x.mean()))[df_data_2.groupby('Metro')[ym].transform(lambda x: x.fillna(x.mean())).isna()]

Series([], Name: 2022-01, dtype: float64)

In [215]:
df_data_2

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01,1996-02,1996-03,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,1997-07,1997-08,1997-09,1997-10,1997-11,1997-12,1998-01,1998-02,1998-03,1998-04,1998-05,1998-06,1998-07,...,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,228165.313018,233969.358296,233133.064945,233443.970861,237353.998041,237486.585262,238150.921270,239538.443050,239793.729378,240699.035363,241930.896414,242230.916226,243306.330079,243374.943609,244168.123100,244800.614066,246075.490814,247681.544210,250038.248818,251547.997078,251334.345944,252624.571633,256025.444268,257593.000051,259983.312856,259983.694731,260806.926778,262810.475357,264848.773670,266633.813405,268861.124317,...,2.792330e+06,2.687480e+06,2.750319e+06,2.745687e+06,2.672237e+06,2.703466e+06,2.799758e+06,2.762261e+06,2.972422e+06,2.964929e+06,2.941399e+06,2.770281e+06,2.709474e+06,2.807152e+06,2.757996e+06,2.898636e+06,2.883489e+06,2.813502e+06,2.835291e+06,2.864990e+06,2.710262e+06,2.851837e+06,2.915106e+06,2.778842e+06,2.812505e+06,2.801768e+06,2.872429e+06,2.869819e+06,2.792227e+06,2.880981e+06,2.936416e+06,3.033201e+06,2.904437e+06,2.801494e+06,2.721834e+06,2.640603e+06,2.616576e+06,2.600348e+06,2.563603e+06,2.729760e+06
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,327184.471500,327713.483700,325984.457900,332778.261000,334414.575100,337601.748300,337856.773200,340445.186300,346222.308200,348798.299600,353450.835200,352990.202800,354387.565100,357231.417600,359681.449200,361963.991400,372552.650700,369575.829700,373462.497000,348818.214100,341442.729200,323433.564700,315459.250600,316542.522300,295871.278300,300335.283900,300666.023500,309593.519800,315575.758500,329702.768100,340326.441500,...,7.892213e+05,7.910212e+05,7.899114e+05,7.805394e+05,7.872967e+05,7.919914e+05,7.900941e+05,7.925490e+05,7.910878e+05,7.864774e+05,7.838324e+05,7.801943e+05,7.750737e+05,7.740944e+05,7.725794e+05,7.801559e+05,7.915183e+05,7.910302e+05,7.915584e+05,7.939067e+05,7.876053e+05,7.908220e+05,7.917972e+05,7.999406e+05,8.017769e+05,7.938478e+05,7.989722e+05,7.971409e+05,8.138223e+05,8.136838e+05,8.204414e+05,8.171122e+05,8.237994e+05,8.301982e+05,8.286601e+05,8.252674e+05,8.381829e+05,8.389180e+05,8.381373e+05,8.495258e+05
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,228165.313018,233969.358296,233133.064945,233443.970861,237353.998041,237486.585262,238150.921270,239538.443050,239793.729378,240699.035363,241930.896414,242230.916226,243306.330079,243374.943609,244168.123100,244800.614066,246075.490814,247681.544210,250038.248818,251547.997078,251334.345944,252624.571633,256025.444268,257593.000051,259983.312856,259983.694731,260806.926778,262810.475357,264848.773670,266633.813405,268861.124317,...,5.306724e+06,5.303907e+06,5.307117e+06,5.156562e+06,5.249348e+06,5.182788e+06,5.058869e+06,5.166357e+06,5.043388e+06,5.041397e+06,4.900863e+06,4.846561e+06,4.806427e+06,4.780078e+06,4.760010e+06,4.874589e+06,4.783694e+06,4.670615e+06,4.821009e+06,4.831602e+06,4.917240e+06,4.823658e+06,4.681429e+06,4.742684e+06,4.750099e+06,4.737670e+06,4.734036e+06,4.503091e+06,4.820726e+06,5.032885e+06,5.072752e+06,5.104681e+06,4.991198e+06,5.204816e+06,5.430261e+06,5.489555e+06,5.183702e+06,5.143927e+06,5.115112e+06,5.111999e+06
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,178597.732900,182239.371600,181934.987000,179668.454700,184266.487000,183117.240300,188706.164800,183779.998300,186036.381900,182704.391400,184768.129800,187562.785700,

In [206]:
df_data_2.describe()

,RegionID,SizeRank,RegionName,1996-01,1996-02,1996-03,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,1997-07,1997-08,1997-09,1997-10,1997-11,1997-12,1998-01,1998-02,1998-03,1998-04,1998-05,1998-06,1998-07,1998-08,1998-09,1998-10,1998-11,1998-12,1999-01,...,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01
count,31990.000000,31990.000000,31990.000000,2.687100e+04,2.700800e+04,2.700800e+04,2.703100e+04,2.713300e+04,2.713900e+04,2.713900e+04,2.715800e+04,2.715800e+04,2.717100e+04,2.719800e+04,2.719800e+04,2.738300e+04,2.738600e+04,2.740100e+04,2.744700e+04,2.744700e+04,2.744700e+04,2.748100e+04,2.752300e+04,2.752300e+04,2.752300e+04,2.755200e+04,2.757600e+04,2.759300e+04,2.759300e+04,2.760200e+04,2.785900e+04,2.797800e+04,2.797800e+04,2.797800e+04,2.800500e+04,2.800800e+04,2.800800e+04,2.800800e+04,2.800800e+04,2.801100e+04,...,3.198100e+04,3.198100e+04,3.198100e+04,3.198100e+04,3.198100e+04,3.198100e+04,3.198100e+04,3.198100e+04,3.198100e+04,3.198100e+04,3.198100e+04,3.198100e+04,3.198100e+04,3.198100e+04,3.198200e+04,3.198200e+04,3.198400e+04,3.198400e+04,3.198400e+04,3.198400e+04,3.198400e+04,3.198400e+04,3.198500e+04,3.198500e+04,3.198500e+04,3.198500e+04,3.198500e+04,3.198600e+04,3.198600e+04,3.198600e+04,3.198600e+04,3.198800e+04,3.198800e+04,3.198800e+04,3.199000e+04,3.199000e+04,3.199000e+04,3.199000e+04,3.199000e+04,3.199000e+04
mean,80501.185433,16531.073367,49095.677618,1.121055e+05,1.128539e+05,1.128168e+05,1.130929e+05,1.133988e+05,1.134923e+05,1.136171e+05,1.137230e+05,1.141217e+05,1.141351e+05,1.142361e+05,1.145100e+05,1.147133e+05,1.147778e+05,1.151423e+05,1.154671e+05,1.161422e+05,1.166275e+05,1.172591e+05,1.174062e+05,1.178297e+05,1.181029e+05,1.186605e+05,1.190135e+05,1.192646e+05,1.195508e+05,1.199012e+05,1.202274e+05,1.212650e+05,1.219889e+05,1.224415e+05,1.231022e+05,1.240768e+05,1.246313e+05,1.251174e+05,1.255521e+05,1.254683e+05,...,2.437459e+05,2.442119e+05,2.447013e+05,2.450688e+05,2.458468e+05,2.468955e+05,2.481033e+05,2.493491e+05,2.500537e+05,2.495971e+05,2.500158e+05,2.505732e+05,2.506750e+05,2.505139e+05,2.514457e+05,2.525903e+05,2.535849e+05,2.551996e+05,2.562373e+05,2.561178e+05,2.572484e+05,2.589269e+05,2.607823e+05,2.632415e+05,2.661095e+05,2.689376e+05,2.712091e+05,2.753901e+05,2.791416e+05,2.829609e+05,2.876937e+05,2.937205e+05,2.992792e+05,3.036842e+05,3.078494e+05,3.105331e+05,3.132967e+05,3.162337e+05,3.224347e+05,3.263482e+05
std,25066.553440,9795.185681,27332.440133,8.140742e+04,8.778045e+04,8.698782e+04,8.771735e+04,9.085660e+04,9.172608e+04,9.245547e+04,9.383929e+04,9.435187e+04,9.509700e+04,9.597416e+04,9.623432e+04,9.566365e+04,9.477706e+04,9.500589e+04,9.497931e+04,9.609366e+04,9.761478e+04,9.974350e+04,1.010426e+05,1.003123e+05,1.011531e+05,1.024774e+05,1.026662e+05,1.031970e+05,1.039242e+05,1.048535e+05,1.071972e+05,1.096746e+05,1.119999e+05,1.147371e+05,1.138073e+05,1.162170e+05,1.168833e+05,1.184410e+05,1.200117e+05,1.182518e+05,...,3.088199e+05,3.080796e+05,3.071565e+05,3.055129e+05,3.054677e+05,3.063888e+05,3.068222e+05,3.091493e+05,3.096893e+05,3.053152e+05,3.043795e+05,3.040810e+05,3.028974e+05,3.023564e+05,3.042708e+05,3.074463e+05,3.087064e+05,3.101690e+05,3.113936e+05,3.081417e+05,3.097596e+05,3.102706e+05,3.123470e+05,3.147815e+05,3.166279e+05,3.168792e+05,3.199028e+05,3.203350e+05,3.248068e+05,3.284824e+05,3.355510e+05,3.441693e+05,3.502804e+05,3.569769e+05,3.627515e+05,3.653355e+05,3.684860e+05,3.721418e+05,3.768795e+05,3.831451e+05
min,58001.000000,0.000000,501.000000,9.590703e+03,9.479815e+03,9.556451e+03,9.628464e+03,9.370827e+03,9.089543e+03,8.598643e+03,8.423374e+03,8.534906e+03,8.144440e+03,7.532039e+03,7.952085e+03,7.919122e+0

In [207]:
df_data_2.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01,1996-02,1996-03,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,1997-07,1997-08,1997-09,1997-10,1997-11,1997-12,1998-01,1998-02,1998-03,1998-04,1998-05,1998-06,1998-07,...,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,228165.313018,233969.358296,233133.064945,233443.970861,237353.998041,237486.585262,238150.92127,239538.44305,239793.729378,240699.035363,241930.896414,242230.916226,243306.330079,243374.943609,244168.1231,244800.614066,246075.490814,247681.54421,250038.248818,251547.997078,251334.345944,252624.571633,256025.444268,257593.000051,259983.312856,259983.694731,260806.926778,262810.475357,264848.77367,266633.813405,268861.124317,...,2.792330e+06,2.687480e+06,2.750319e+06,2.745687e+06,2.672237e+06,2.703466e+06,2.799758e+06,2.762261e+06,2.972422e+06,2.964929e+06,2.941399e+06,2.770281e+06,2.709474e+06,2.807152e+06,2.757996e+06,2.898636e+06,2.883489e+06,2.813502e+06,2.835291e+06,2.864990e+06,2.710262e+06,2.851837e+06,2.915106e+06,2.778842e+06,2.812505e+06,2.801768e+06,2.872429e+06,2869819.021,2.792227e+06,2.880981e+06,2.936416e+06,3.033201e+06,2.904437e+06,2.801494e+06,2.721834e+06,2.640603e+06,2.616576e+06,2.600348e+06,2.563603e+06,2.729760e+06
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,327184.471500,327713.483700,325984.457900,332778.261000,334414.575100,337601.748300,337856.77320,340445.18630,346222.308200,348798.299600,353450.835200,352990.202800,354387.565100,357231.417600,359681.4492,361963.991400,372552.650700,369575.82970,373462.497000,348818.214100,341442.729200,323433.564700,315459.250600,316542.522300,295871.278300,300335.283900,300666.023500,309593.519800,315575.75850,329702.768100,340326.441500,...,7.892213e+05,7.910212e+05,7.899114e+05,7.805394e+05,7.872967e+05,7.919914e+05,7.900941e+05,7.925490e+05,7.910878e+05,7.864774e+05,7.838324e+05,7.801943e+05,7.750737e+05,7.740944e+05,7.725794e+05,7.801559e+05,7.915183e+05,7.910302e+05,7.915584e+05,7.939067e+05,7.876053e+05,7.908220e+05,7.917972e+05,7.999406e+05,8.017769e+05,7.938478e+05,7.989722e+05,797140.864,8.138223e+05,8.136838e+05,8.204414e+05,8.171122e+05,8.237994e+05,8.301982e+05,8.286601e+05,8.252674e+05,8.381829e+05,8.389180e+05,8.381373e+05,8.495258e+05
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,228165.313018,233969.358296,233133.064945,233443.970861,237353.998041,237486.585262,238150.92127,239538.44305,239793.729378,240699.035363,241930.896414,242230.916226,243306.330079,243374.943609,244168.1231,244800.614066,246075.490814,247681.54421,250038.248818,251547.997078,251334.345944,252624.571633,256025.444268,257593.000051,259983.312856,259983.694731,260806.926778,262810.475357,264848.77367,266633.813405,268861.124317,...,5.306724e+06,5.303907e+06,5.307117e+06,5.156562e+06,5.249348e+06,5.182788e+06,5.058869e+06,5.166357e+06,5.043388e+06,5.041397e+06,4.900863e+06,4.846561e+06,4.806427e+06,4.780078e+06,4.760010e+06,4.874589e+06,4.783694e+06,4.670615e+06,4.821009e+06,4.831602e+06,4.917240e+06,4.823658e+06,4.681429e+06,4.742684e+06,4.750099e+06,4.737670e+06,4.734036e+06,4503091.435,4.820726e+06,5.032885e+06,5.072752e+06,5.104681e+06,4.991198e+06,5.204816e+06,5.430261e+06,5.489555e+06,5.183702e+06,5.143927e+06,5.115112e+06,5.111999e+06
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,178597.732900,182239.371600,181934.987000,179668.454700,184266.487000,183117.240300,188706.16480,183779.99830,186036.381900,182704.391400,184768.129800,187562.785700,189633.275100,189402.063

In [208]:
# Columns that want to be kept
to_keep = ['RegionName', 'State']

# Columns to be converted to observations
to_obs = df_data_2.iloc[:,9:]

# Pivot the data to have on observation for each region for each year-month. 
df_data_pivot = pd.pivot_table(df_data_2, columns=to_keep, values=to_obs)

# Reset the index and rename year-month and values columns for easy identification.
df_data_pivot = df_data_pivot.reset_index()
df_data_pivot.rename(columns={'level_0':'date', 0:'value'}, inplace=True)

# Convert data column to datetime
df_data_pivot['index'] = pd.to_datetime(df_data_pivot['index'], infer_datetime_format=True)

# Print first five rows
df_data_pivot.sort_values(['index']).head()

RegionName,index,501,601,602,605,606,612,614,616,662,693,705,717,725,734,740,745,765,791,803,822,902,914,917,924,926,949,953,956,957,971,985,1001,1002,1005,1007,1008,1009,1010,1011,...,99587,99603,99605,99610,99611,99615,99623,99631,99635,99639,99643,99644,99645,99652,99654,99663,99664,99669,99672,99674,99676,99683,99686,99687,99688,99701,99705,99709,99712,99714,99721,99723,99747,99759,99782,99789,99801,99824,99833,99901
State,,NY,MS,AL,AL,MO,MI,AR,TX,AL,PA,UT,TX,UT,TX,TX,AR,CO,NM,TX,LA,UT,MS,UT,PA,LA,NM,LA,LA,KY,AR,AR,MA,MA,MA,MA,MA,TX,MA,MA,...,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK,AK
0,1996-01-01,228165.313018,NaN,154415.831725,NaN,NaN,NaN,NaN,NaN,154415.831725,76321.756097,NaN,84660.491696,NaN,84660.491696,139537.322827,47721.949575,NaN,NaN,NaN,58540.807453,NaN,NaN,NaN,76321.756097,NaN,NaN,NaN,NaN,129190.101382,NaN,NaN,118460.7970,176567.1996,112310.0350,141597.4563,123288.8251,NaN,127846.4124,102799.5820,...,222360.3936,NaN,NaN,NaN,NaN,NaN,136951.529019,NaN,NaN,NaN,NaN,NaN,136951.529019,136951.529019,136951.529019,NaN,NaN,NaN,NaN,136951.529019,136951.529019,136951.529019,NaN,136951.529019,136951.529019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1996-02-01,233969.358296,NaN,152604.043452,NaN,NaN,NaN,NaN,NaN,152604.043452,76097.742469,NaN,85704.381716,NaN,85704.381716,138909.918989,47621.074341,NaN,NaN,NaN,58548.600440,NaN,NaN,NaN,76097.742469,NaN,NaN,NaN,NaN,130150.793886,NaN,NaN,118631.9315,176586.9652,111865.4568,141719.0672,123423.5183,NaN,128266.8602,102435.7249,...,225706.1527,NaN,NaN,NaN,NaN,NaN,138930.159885,NaN,NaN,NaN,NaN,NaN,138930.159885,138930.159885,138930.159885,NaN,NaN,NaN,NaN,138930.159885,138930.159885,138930.159885,NaN,138930.159885,138930.159885,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1996-03-01,233133.064945,NaN,152996.856036,NaN,NaN,NaN,NaN,NaN,152996.856036,76106.735737,NaN,86368.710706,NaN,86368.710706,139732.023709,47854.233298,NaN,NaN,NaN,58812.348617,NaN,NaN,NaN,76106.735737,NaN,NaN,NaN,NaN,130187.044444,NaN,NaN,120127.3008,177257.3589,111644.1498,141981.7976,123533.9928,NaN,128755.5195,103124.3276,...,214580.2271,NaN,NaN,NaN,NaN,NaN,137875.453528,NaN,NaN,NaN,NaN,NaN,137875.453528,137875.453528,137875.453528,NaN,NaN,NaN,NaN,137875.453528,137875.453528,137875.453528,NaN,137875.453528,137875.453528,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1996-04-01,233443.970861,NaN,152611.516944,NaN,NaN,NaN,NaN,NaN,152611.516944,76054.855997,NaN,86911.730028,NaN,86911.730028,140416.803698,48365.127041,NaN,NaN,NaN,58806.895067,NaN,NaN,NaN,76054.855997,NaN,NaN,NaN,NaN,131116.885848,NaN,NaN,120009.2743,177295.6388,113731.5806,142401.4240,122392.7136,NaN,128502.6604,102215.4506,...,225056.6084,NaN,NaN,NaN,NaN,NaN,141226.430248,NaN,NaN,NaN,NaN,NaN,141226.430248,141226.430248,141226.430248,NaN,NaN,NaN,NaN,141226.430248,141226.430248,141226.430248,NaN,141226.430248,141226.430248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1996-05-01,237353.998041,NaN,153249.023618,NaN,NaN,NaN,NaN,NaN,153249.023618,76315.791089,NaN,87785.775352,NaN,87785.775352,140493.408448,48586.659804,NaN,NaN,NaN,59375.902610,NaN,NaN,NaN,76315.791089,NaN,NaN,NaN,NaN,131782.488875,NaN,NaN,118897.6000,177442.8574,114118.0097,143436.1103,123007.2410,NaN,128861.0318,104023.7548,...,219638.1123,NaN,NaN,NaN,NaN,NaN,139737.917972,NaN,NaN,NaN,NaN,NaN,139737.917972,139737.917972,139737.917972,NaN,NaN,NaN,NaN,139737.917972,139737.917972,139737.917972,NaN,139737.917972,139737.917972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [188]:
print(df_data_pivot.shape)
print(df_data_pivot.dtypes)

(313, 31991)
RegionName  State
index                datetime64[ns]
501         NY              float64
601         MS              float64
602         AL              float64
605         AL              float64
                          ...      
99789       AK              float64
99801       AK              float64
99824       AK              float64
99833       AK              float64
99901       AK              float64
Length: 31991, dtype: object


In [163]:
df_data_pivot.head()

RegionName,index,602,612,662,693,717,725,745,791,917,924,926,949,953,956,957,985,1001,1002,1013,1020,1027,1040,1056,1060,1075,1085,1089,1104,1108,1109,1201,1247,1301,1420,1440,1453,1501,1545,1550,...,98942,98944,99004,99016,99037,99163,99201,99202,99203,99205,99206,99207,99208,99212,99216,99217,99223,99224,99301,99336,99337,99352,99354,99362,99403,99501,99502,99504,99507,99508,99515,99516,99517,99577,99645,99654,99701,99705,99709,99801
City,,Auburn,Frederic,Isabela,Greensburg,Ponce,Caguas,Rio Grande,Humacao,San Juan,Derry Township,Zwolle,Candelaria Arenas,Ethel,Ethel,Midway,Berryville,Agawam,Amherst,Chicopee,Chicopee,Easthampton,Holyoke,Ludlow,Northampton,South Hadley,Westfield,West Springfield,Springfield,Springfield,Springfield,Pittsfield,North Adams,Greenfield,Fitchburg,Gardner,Leominster,Auburn,Shrewsbury,Southbridge,...,Selah,Sunnyside,Cheney,Spokane Valley,Spokane Valley,Pullman,Spokane,Spokane,Spokane,Spokane,Spokane Valley,Spokane,Spokane,Spokane Valley,Spokane Valley,Spokane,Spokane,Spokane,Pasco,Kennewick,Kennewick,Richland,Richland,Walla Walla,Clarkston,Anchorage,Anchorage,Anchorage,Anchorage,Anchorage,Anchorage,Anchorage,Anchorage,Anchorage,Palmer,Wasilla,Fairbanks,North Pole,Fairbanks,Juneau
0,1996-01-01,133836.195920,NaN,133836.195920,99311.927966,118871.16320,NaN,87861.913710,NaN,NaN,99311.927966,NaN,NaN,NaN,NaN,111640.650522,NaN,118460.7970,176567.1996,92222.24066,99265.50514,125310.4606,97882.03284,76705.14266,173482.8798,133205.1592,118595.7594,116442.1398,72195.88025,81958.14658,69102.11507,100739.4438,85884.43218,97151.52707,96652.21313,93410.46956,120685.8848,122173.7232,122276.3707,100961.5901,...,NaN,NaN,138229.3319,135297.9245,163501.5428,NaN,100005.23540,86190.21102,147122.7637,91239.10484,130583.0190,79469.30804,152633.3836,111279.4364,115363.2879,121482.0128,167978.4131,181324.6826,99344.21478,99344.21478,99344.21478,99344.21478,99344.21478,103754.5034,NaN,111497.5702,136391.5939,115553.1551,128085.2878,113521.0047,160218.4172,202814.2105,136047.6792,153787.7347,139768.517033,139768.517033,NaN,NaN,NaN,NaN
1,1996-02-01,133582.933764,NaN,133582.933764,98757.503739,120137.09140,NaN,87677.628965,NaN,NaN,98757.503739,NaN,NaN,NaN,NaN,112130.318164,NaN,118631.9315,176586.9652,92324.74278,100160.44150,124936.5505,97890.13069,76651.27340,172471.6292,131948.5165,118947.3470,115792.3150,72389.02606,81523.80084,68551.28097,100654.5498,85925.12917,96738.26443,97609.23712,94659.49182,121481.2344,121877.1472,123003.4604,101728.6833,...,NaN,NaN,139771.6618,135969.5236,163946.0110,NaN,99931.95901,85668.13469,147368.5818,90889.38082,130693.8609,79428.11401,152428.7049,111585.0366,115337.6139,121683.2560,168190.9492,180377.0520,99660.78622,99660.78622,99660.78622,99660.78622,99660.78622,103714.6144,NaN,112380.7952,136921.0956,115842.2668,130240.2083,113699.3863,164020.0674,210636.9163,136091.8321,154926.3110,141639.875444,141639.875444,NaN,NaN,NaN,NaN
2,1996-03-01,133638.841798,NaN,133638.841798,98520.811167,120724.53440,NaN,88518.377760,NaN,NaN,98520.811167,NaN,NaN,NaN,NaN,112231.859983,NaN,120127.3008,177257.3589,93682.74072,101155.08710,125176.5801,98403.15063,76756.12879,174918.6078,131783.8134,119415.0912,115502.2480,71886.22332,82472.25253,69788.44746,100524.8282,85973.75569,96850.60515,97164.13835,93726.74033,121423.8724,121677.2799,122239.9318,101163.5030,...,NaN,NaN,141123.8466,136522.8509,163500.8042,NaN,99333.14876,85039.44107,145923.5672,89817.00991,130261.3757,78154.81424,151124.2300,111723.1164,114891.6427,121322.6430,167401.6169,180918.1522,99999.35050,99999.35050,99999.35050,99999.35050,99999.35050,103725.6282,NaN,114390.7594,136756.5611,119752.0433,129487.3143,115748.0570,161676.3031,207390.0191,135767.4738,160928.7327,142433.029311,142433.029311,NaN,NaN,NaN,NaN
3,1996-04-01,133335.193510,NaN,133335.193510,98377.927586,121754.14935,NaN,87543.770430,NaN,NaN,98377.927586,NaN,NaN,NaN,NaN,112436.060105,NaN,120009.2743,177295.6388,92978.10063,101628.88660,125541.7073,98544.57867,77212.47977,173250.6682,1314

In [164]:
# Print the first and last date of the data 
print('Oldest observation: ', df_data_pivot['index'].min())
print('Newest observation: ', df_data_pivot['index'].max(), '\n')

Oldest observation:  1996-01-01 00:00:00
Newest observation:  2022-01-01 00:00:00 

